In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [2]:
questions = {'type1':[] , 'type2':[] , 'type3':[]}
data_file = open('../database/intents1.json').read()
intents = json.loads(data_file)
for i in intents['type1']:
    for question in i['patterns']:
        if question != '':
            questions['type1'].append(question)
        
data_file = open('../database/intents2.json').read()
intents = json.loads(data_file)
for i in intents['type2']:
    for question in i['patterns']:
        if question != '':
            questions['type2'].append(question)
        
data_file = open('../database/intents3.json').read()
intents = json.loads(data_file)
for i in intents['type3']:
    for question in i['patterns']:
        if question != '':
            questions['type3'].append(question)

In [4]:
words=[]
classes = ['type1' , 'type2' , 'type3']
documents = []
ignore_words = ['?', '!']

for qtype in questions:
    for question in questions[qtype]:
        w = nltk.word_tokenize(question)
        words.extend(w)
        documents.append((w, qtype))

In [5]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [6]:
# Classes has list of intents
# Documents : (['words','in', 'the' , 'message'] , 'intent')
# Words : All words in sorted order

In [7]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
    
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-7-32163f03cfcd>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [11]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('../models/model.h5', hist)

print("model created")

Epoch 1/200
50/50 [==============================] - 0s 972us/step - loss: 0.9268 - accuracy: 0.5360
Epoch 2/200
50/50 [==============================] - 0s 974us/step - loss: 0.7004 - accuracy: 0.7080
Epoch 3/200
50/50 [==============================] - 0s 1ms/step - loss: 0.5279 - accuracy: 0.7880
Epoch 4/200
50/50 [==============================] - 0s 880us/step - loss: 0.4331 - accuracy: 0.8360
Epoch 5/200
50/50 [==============================] - 0s 856us/step - loss: 0.2935 - accuracy: 0.8720
Epoch 6/200
50/50 [==============================] - 0s 873us/step - loss: 0.2266 - accuracy: 0.9400
Epoch 7/200
50/50 [==============================] - 0s 961us/step - loss: 0.1826 - accuracy: 0.9240
Epoch 8/200
50/50 [==============================] - 0s 988us/step - loss: 0.1570 - accuracy: 0.9520
Epoch 9/200
50/50 [==============================] - 0s 1ms/step - loss: 0.1347 - accuracy: 0.9440
Epoch 10/200
50/50 [==============================] - 0s 891us/step - loss: 0.1196 - accuracy: 

50/50 [==============================] - 0s 1ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 83/200
50/50 [==============================] - 0s 960us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 84/200
50/50 [==============================] - 0s 969us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 85/200
50/50 [==============================] - 0s 988us/step - loss: 0.0030 - accuracy: 1.0000
Epoch 86/200
50/50 [==============================] - 0s 1ms/step - loss: 0.0182 - accuracy: 0.9920
Epoch 87/200
50/50 [==============================] - 0s 1ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 88/200
50/50 [==============================] - 0s 1ms/step - loss: 0.0091 - accuracy: 0.9960
Epoch 89/200
50/50 [==============================] - 0s 1ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 90/200
50/50 [==============================] - 0s 1ms/step - loss: 0.0111 - accuracy: 0.9960
Epoch 91/200
50/50 [==============================] - 0s 990us/step - loss: 0.0038 - accuracy: 1.0000
Epoch

50/50 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 163/200
50/50 [==============================] - 0s 1ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 164/200
50/50 [==============================] - 0s 953us/step - loss: 0.0020 - accuracy: 1.0000
Epoch 165/200
50/50 [==============================] - 0s 923us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 166/200
50/50 [==============================] - 0s 952us/step - loss: 7.8604e-04 - accuracy: 1.0000
Epoch 167/200
50/50 [==============================] - 0s 941us/step - loss: 1.9854e-04 - accuracy: 1.0000
Epoch 168/200
50/50 [==============================] - 0s 876us/step - loss: 7.4803e-04 - accuracy: 1.0000
Epoch 169/200
50/50 [==============================] - 0s 834us/step - loss: 9.4196e-04 - accuracy: 1.0000
Epoch 170/200
50/50 [==============================] - 0s 1ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 171/200
50/50 [==============================] - 0s 1ms/step - loss: 0.